In [ ]:
from brayns import Client
from circuitexplorer import CircuitExplorer
brayns = Client('localhost:5000')
ce = CircuitExplorer(brayns)
config = '/home/favreau/git/install/release/share/BBPTestData/circuitBuilding_1000neurons/BlueConfig'
# config = '/gpfs/bbp.cscs.ch/release/l2/data/circuits/SomatosensoryCxS1-v5.r0/O1/merged_circuit/CircuitConfig'

In [ ]:
''' Single cell '''
model = ce.load_circuit(
    use_sdf=True,
    load_afferent_synapses=True, load_efferent_synapses=True,
    load_soma=True, load_axon=True, load_apical_dendrite=True, load_dendrite=True,
    circuit_color_scheme=ce.CIRCUIT_COLOR_SCHEME_NEURON_BY_MTYPE,
    morphology_color_scheme=ce.MORPHOLOGY_COLOR_SCHEME_NONE,
    density=100, path=config, 
    targets=['mini50']
)

In [ ]:
brayns.set_camera(
    orientation=[0.0, 0.0, 0.0, 1.0],
    position=[48.22725520482774, 1400.2945376573034, 94.8800494769331],
    target=[48.22725520482774, 1400.2945376573034, -509.0384521484375]
)

In [ ]:
def set_materials(model_id, palette_name, opacity=1.0):
    
    simulation_data_casts=list()
    opacities=list()
    refraction_indices=list()
    reflection_indices=list()
    shading_modes=list()
    diffuse_colors=list()
    specular_colors=list()
    specular_exponents=list()
    material_ids=list()
    glossinesses=list()
    emissions=list()
    clips=list()
    user_parameters=list()
    
    import seaborn as sns
    material_ids = ce.get_material_ids(model_id)['ids']
    sns_palette = sns.color_palette(palette_name, len(material_ids))
    palette = list()
    
    i = 0
    for material_id in material_ids:
        c=sns_palette[i]
        mid = material_id % 10
        if mid == ce.MATERIAL_OFFSET_AFFERENT_SYNPASE:
            palette.append([1,1,1])
            emissions.append(1.0)
        elif mid == ce.MATERIAL_OFFSET_EFFERENT_SYNPASE:
            palette.append([1,1,1])
            emissions.append(1.0)
        else:
            palette.append(c)
            emissions.append(0)
        i += 1
    
    for i in range(len(material_ids)):
        c=sns_palette[i]
        opacities.append(opacity)
        refraction_indices.append(1.0)
        reflection_indices.append(0.0)
        specular_exponents.append(3)
        glossinesses.append(0.7)
        diffuse_colors.append([c[0],c[1],c[2]])
        specular_colors.append([c[0],c[1],c[2]])
        shading_modes.append(ce.SHADING_MODE_PERLIN)
        simulation_data_casts.append(False)
        user_parameters.append(0.08)
        clips.append(False)
        
    ce.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        simulation_data_casts=simulation_data_casts,
        opacities=opacities, reflection_indices=reflection_indices,
        shading_modes=shading_modes, user_parameters=user_parameters,
        diffuse_colors=diffuse_colors, specular_colors=specular_colors,
        specular_exponents=specular_exponents, glossinesses=glossinesses,
        emissions=emissions, refraction_indices=refraction_indices)
    
model_id = model['id']
ce.set_material_extra_attributes(model_id)
set_materials(model_id, 'rainbow', 1.0)

In [ ]:
brayns.set_renderer(subsampling=4, max_accum_frames=256)